<a href="https://colab.research.google.com/github/alvinfranklyndavis/Project2023_v3/blob/main/Data_Prep_GPT_4_Bard_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CELL 1.1: Package Installation and Library Import

# Upgrade pip and install required packages
!pip install -U --upgrade-strategy eager pip
!pip install -U --upgrade-strategy eager pandas numpy

# Import required libraries
import pandas as pd
import numpy as np
import logging
import os

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [15]:
# Cell 1.2: Data Loading from Google Drive and preprocessing Training / Testing dataset

import pandas as pd
import logging
import os
from google.colab import drive

# Set up logging
logger = logging.getLogger(__name__)

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory for datasets in Google Drive
drive_dataset_directory = '/content/drive/My Drive/Predictive_Modeling_Four_Draws/Morning_Draw_Model_Docs/'

# Define the path to the comprehensive CSV file for training and testing
csv_filename_train_test = 'Model_Train_Test_Data.csv'
drive_csv_path_train_test = os.path.join(drive_dataset_directory, csv_filename_train_test)

# Check and load the datasets
def load_dataset(file_path):
    if os.path.isfile(file_path):
        print("File found. Proceeding to load the dataset.")
        return pd.read_csv(file_path)
    else:
        print("File not found. Check the file path or the Google Drive mount.")
        return None

train_test_data = load_dataset(drive_csv_path_train_test)

# Function to preprocess training/testing data
def preprocess_train_test_data(data):
    print("Initial data columns:", data.columns)

    # Convert 'Date' to datetime and extract 'Year', 'Month', and 'Day'
    if 'Date' in data.columns:
        print("Converting 'Date' to datetime...")
        data['Date'] = pd.to_datetime(data['Date'])
        data['Year'] = data['Date'].dt.year
        data['Month'] = data['Date'].dt.month
        data['Day'] = data['Date'].dt.day
        print("After extracting Year, Month, Day:", data.columns)
        data.drop(columns=['Date'], inplace=True)
        print("After dropping 'Date':", data.columns)
    else:
        print("Date column not found in the given dataset.")

    # Initialize 'Prediction1' column with 'Morning' values
    data['Prediction1'] = data['Morning']

    # Create shifted columns for previous day's data
    data['Prev_Morning'] = data['Morning'].shift(1)
    data['Prev_Afternoon'] = data['Afternoon'].shift(1)
    data['Prev_Evening'] = data['Evening'].shift(1)

    # Calculate moving averages excluding current row
    initial_window_size = 3  # Increased by 1 to exclude the current row
    columns_to_average = ['Morning', 'Afternoon', 'Evening', 'Night']
    target_columns = ['Mov_Avg_Mor', 'Mov_Avg_Aft', 'Mov_Avg_Eve', 'Mov_Avg_Nig']

    for col, target_col in zip(columns_to_average, target_columns):
    # Roll over an additional row and then shift to exclude the current row
        data[target_col] = data[col].rolling(window=initial_window_size, min_periods=1).mean().shift(1)

    # Calculate vertical averages excluding current row
    vertical_target_columns = ['Vert_Avg_Mor', 'Vert_Avg_Aft', 'Vert_Avg_Eve', 'Vert_Avg_Nig']
    for col, target_col in zip(columns_to_average, vertical_target_columns):
        data[target_col] = data[col].rolling(window=3, min_periods=1).mean().shift(1)

    # Handle NaN values
    data['Prev_Morning'].fillna(13, inplace=True)
    data['Prev_Afternoon'].fillna(34, inplace=True)
    data['Prev_Evening'].fillna(32, inplace=True)
    data['Mov_Avg_Mor'].fillna(27.5, inplace=True)
    data['Mov_Avg_Aft'].fillna(21, inplace=True)
    data['Mov_Avg_Eve'].fillna(16.5, inplace=True)
    data['Mov_Avg_Nig'].fillna(23.5, inplace=True)
    data['Vert_Avg_Mor'].fillna(10, inplace=True)
    data['Vert_Avg_Aft'].fillna(23.5, inplace=True)
    data['Vert_Avg_Eve'].fillna(30, inplace=True)
    data['Vert_Avg_Nig'].fillna(12.5, inplace=True)

    # Select relevant columns, including 'Prediction1'
    selected_columns = ['Row Number', 'Data_Type', 'Year', 'Month', 'Day', 'Prev_Week', 'Prev_Entry', 'Mov_Avg_Mor', 'Vert_Avg_Mor', 'Prev_Morning', 'Prev_Afternoon', 'Prev_Evening', 'Prediction1']
    data[selected_columns]

    return data

# Apply preprocessing to the training/testing dataset
train_test_data = preprocess_train_test_data(train_test_data)

# Display the preprocessed data
print("First few rows of preprocessed training/testing data:")
print(train_test_data.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File found. Proceeding to load the dataset.
Initial data columns: Index(['Date', 'Row Number', 'Data_Type', 'Morning', 'Prev_Week',
       'Rep_Prev_Week', 'Prev_Entry', 'Rep_Prev_Entry', 'Mov_Avg_Mor',
       'Afternoon', 'Prev_Week.1', 'Rep_Prev_Week.1', 'Prev_Entry.1',
       'Rep_Prev_Entry.1', 'Mov_Avg_Aft', 'Evening', 'Prev_Week.2',
       'Rep_Prev_Week.2', 'Prev_Entry.2', 'Rep_Prev_Entry.2', 'Mov_Avg_Eve',
       'Night', 'Prev_Week.3', 'Rep_Prev_Week.3', 'Prev_Entry.3',
       'Rep_Prev_Entry.3', 'Mov_Avg_Nig'],
      dtype='object')
Converting 'Date' to datetime...
After extracting Year, Month, Day: Index(['Date', 'Row Number', 'Data_Type', 'Morning', 'Prev_Week',
       'Rep_Prev_Week', 'Prev_Entry', 'Rep_Prev_Entry', 'Mov_Avg_Mor',
       'Afternoon', 'Prev_Week.1', 'Rep_Prev_Week.1', 'Prev_Entry.1',
       'Rep_Prev_Entry.1', 'Mov_Avg_Aft', 'Even

In [16]:
# Cell 1.3: Data Loading from Google Drive and Preprocessing Unseen Dataset

import pandas as pd
import logging
import os
from google.colab import drive

# Set up logging
logger = logging.getLogger(__name__)

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory for datasets in Google Drive
drive_dataset_directory = '/content/drive/My Drive/Predictive_Modeling_Four_Draws/Morning_Draw_Model_Docs/'

# Define the path to the CSV file for unseen data
csv_filename_unseen = 'Model_Unseen_Data.csv'
drive_csv_path_unseen = os.path.join(drive_dataset_directory, csv_filename_unseen)

# Check and load the dataset
def load_dataset(file_path):
    if os.path.isfile(file_path):
        print("File found. Proceeding to load the dataset.")
        return pd.read_csv(file_path)
    else:
        print("File not found. Check the file path or the Google Drive mount.")
        return None

unseen_data = load_dataset(drive_csv_path_unseen)

# Function to preprocess unseen data
def preprocess_unseen_data(data):
    print("Initial data columns:", data.columns)

    # Convert 'Date' to datetime and extract 'Year', 'Month', and 'Day'
    if 'Date' in data.columns:
        print("Converting 'Date' to datetime...")
        data['Date'] = pd.to_datetime(data['Date'])
        data['Year'] = data['Date'].dt.year
        data['Month'] = data['Date'].dt.month
        data['Day'] = data['Date'].dt.day
        print("After extracting Year, Month, Day:", data.columns)
        data.drop(columns=['Date'], inplace=True)
        print("After dropping 'Date':", data.columns)
    else:
        print("Date column not found in the given dataset.")

    # Initialize 'Prediction1' column with NaNs for unseen data
    data['Prediction1'] = np.nan

    # Create shifted columns for previous day's data
    data['Prev_Morning'] = data['Morning'].shift(1)
    data['Prev_Afternoon'] = data['Afternoon'].shift(1)
    data['Prev_Evening'] = data['Evening'].shift(1)

    # Calculate moving averages excluding current row
    initial_window_size = 3  # Increased by 1 to exclude the current row
    columns_to_average = ['Morning', 'Afternoon', 'Evening', 'Night']
    target_columns = ['Mov_Avg_Mor', 'Mov_Avg_Aft', 'Mov_Avg_Eve', 'Mov_Avg_Nig']

    for col, target_col in zip(columns_to_average, target_columns):
    # Roll over an additional row and then shift to exclude the current row
        data[target_col] = data[col].rolling(window=initial_window_size, min_periods=1).mean().shift(1)
    # Manually set the value for the first row
    unseen_data.at[0, 'Mov_Avg_Mor'] = 6
    unseen_data.at[1, 'Mov_Avg_Mor'] = 22
    unseen_data.at[2, 'Mov_Avg_Mor'] = 17.5
    unseen_data.at[3, 'Mov_Avg_Mor'] = 2
    unseen_data.at[4, 'Mov_Avg_Mor'] = 17

    # Calculate vertical averages excluding current row
    vertical_target_columns = ['Vert_Avg_Mor', 'Vert_Avg_Aft', 'Vert_Avg_Eve', 'Vert_Avg_Nig']
    for col, target_col in zip(columns_to_average, vertical_target_columns):
        data[target_col] = data[col].rolling(window=3, min_periods=1).mean().shift(1)

    # Handle NaN values
    data['Prev_Morning'].fillna(25, inplace=True)
    data['Prev_Afternoon'].fillna(9, inplace=True)
    data['Prev_Evening'].fillna(7, inplace=True)

    # Select relevant columns, including 'Prediction1'
    selected_columns = ['Row Number', 'Data_Type', 'Year', 'Month', 'Day', 'Prev_Week', 'Prev_Entry', 'Mov_Avg_Mor', 'Vert_Avg_Mor', 'Prev_Morning', 'Prev_Afternoon', 'Prev_Evening', 'Prediction1']
    data[selected_columns]

    return data

# Apply preprocessing to the unseen dataset
unseen_data = preprocess_unseen_data(unseen_data)

# Display the preprocessed unseen data
print("First few rows of preprocessed unseen data:")
print(unseen_data.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File found. Proceeding to load the dataset.
Initial data columns: Index(['Date', 'Row Number', 'Data_Type', 'Morning', 'Prev_Week',
       'Rep_Prev_Week', 'Prev_Entry', '2WeeksM', 'Mov_Avg_Mor', 'Afternoon',
       'Prev_Week.1', 'Rep_Prev_Week.1', 'Prev_Entry.1', '2WeeksA',
       'Mov_Avg_Aft', 'Evening', 'Prev_Week.2', 'Rep_Prev_Week.2',
       'Prev_Entry.2', '2WeeksE', 'Mov_Avg_Eve', 'Night', 'Prev_Week.3',
       'Rep_Prev_Week.3', 'Prev_Entry.3', '2WeeksN', 'Mov_Avg_Nig'],
      dtype='object')
Converting 'Date' to datetime...
After extracting Year, Month, Day: Index(['Date', 'Row Number', 'Data_Type', 'Morning', 'Prev_Week',
       'Rep_Prev_Week', 'Prev_Entry', '2WeeksM', 'Mov_Avg_Mor', 'Afternoon',
       'Prev_Week.1', 'Rep_Prev_Week.1', 'Prev_Entry.1', '2WeeksA',
       'Mov_Avg_Aft', 'Evening', 'Prev_Week.2', 'Rep_Prev_Week.2',
       'Prev_Entr

In [21]:
# Cell 1.4: # Save the preprocessed training/testing dataset
preprocessed_train_test_path = os.path.join(drive_dataset_directory, '15_preprocessed_train_test_data.csv')
train_test_data.to_csv(preprocessed_train_test_path, index=False)
print("Preprocessed training/testing data saved to Google Drive.")

# Display the first few rows of the preprocessed training/testing data
print("First few rows of preprocessed training/testing data:")
print(train_test_data.head())

# Check for NaN values in the entire dataset
nan_counts = train_test_data.isnull().sum()
print("Count of NaN values in training/testing data:")
print(nan_counts)

# Save the preprocessed unseen dataset
preprocessed_unseen_path = os.path.join(drive_dataset_directory, '16_preprocessed_unseen_data.csv')
unseen_data.to_csv(preprocessed_unseen_path, index=False)
print("Preprocessed unseen data saved to Google Drive.")

# Display the first few rows of the preprocessed unseen data
print("First few rows of preprocessed unseen data:")
print(unseen_data.head())

# Check for NaN values in the entire dataset
nan_counts = unseen_data.isnull().sum()
print("Count of NaN values in unseen data:")
print(nan_counts)


Preprocessed training/testing data saved to Google Drive.
First few rows of preprocessed training/testing data:
   Row Number Data_Type  Morning  Prev_Week  Rep_Prev_Week  Prev_Entry  \
0           1  Training       19          7              0          23   
1           2  Training       31         11              0           9   
2           3  Training       15         19              0          12   
3           4  Training       31         35              0          35   
4           5  Training       31         18              0          16   

   Rep_Prev_Entry  Mov_Avg_Mor  Afternoon  Prev_Week.1  ...  Month  Day  \
0               0    27.500000         14           13  ...      8    1   
1               0    19.000000          3           21  ...      8    2   
2               0    25.000000          9           19  ...      8    3   
3               0    21.666667         21           20  ...      8    4   
4               0    25.666667         31           30  ...      8  

In [43]:
import pandas as pd
import logging
import os

# Set up logging
logger = logging.getLogger(__name__)

# Define the directory for datasets in Google Drive
drive_dataset_directory = '/content/drive/My Drive/Predictive_Modeling_Four_Draws/Morning_Draw_Model_Docs/'

# Define the path to the preprocessed unseen data
preprocessed_unseen_path = os.path.join(drive_dataset_directory, '16_preprocessed_unseen_data.csv')

# Load the preprocessed unseen data
unseen_data = pd.read_csv(preprocessed_unseen_path)
logger.info("Preprocessed unseen data loaded successfully.")

# Define the provided data for imputation
provided_data = [
    {
        'Row Number': 1410,
        'Morning': 13,
        'Prev_Week': 27,
        '2WeeksM': 25,
        'Prev_Entry': 5,
        'Prev_Entry-2': 7,
        'Mov_Avg_Mor': 6,
        'Vert_Avg_Mor': 26,
        'Afternoon': 20,
        'Prev_Week': 7,
        '2WeeksA': 34,
        'Prev_Entry': 13,
        'Prev_Entry-2': 5,
        'Mov_Avg_Aft': 9,
        'Vert_Avg_Aft': 20.5,
        'Evening': 26,
        'Prev_Week': 26,
        '2WeeksE': 24,
        'Prev_Entry': 20,
        'Prev_Entry-2': 13,
        'Mov_Avg_Eve': 16.5,
        'Vert_Avg_Eve': 25,
        'Night': 18,
        'Prev_Week': 26,
        '2WeeksN': 3,
        'Prev_Entry': 26,
        'Prev_Entry-2': 20,
        'Mov_Avg_Nig': 23,
        'Vert_Avg_Nig': 14.5
    },
    {
        'Row Number': 1411,
        'Morning': 21,
        'Prev_Week': 33,
        '2WeeksM': 12,
        'Prev_Entry': 18,
        'Prev_Entry-2': 26,
        'Mov_Avg_Mor': 22,
        'Vert_Avg_Mor': 22.5,
        'Afternoon': 31,
        'Prev_Week': 18,
        '2WeeksA': 36,
        'Prev_Entry': 21,
        'Prev_Entry-2': 18,
        'Mov_Avg_Aft': 19.5,
        'Vert_Avg_Aft': 27,
        'Evening': 7,
        'Prev_Week': 9,
        '2WeeksE': 3,
        'Prev_Entry': 31,
        'Prev_Entry-2': 21,
        'Mov_Avg_Eve': 26,
        'Vert_Avg_Eve': 6,
        'Night': 28,
        'Prev_Week': 8,
        '2WeeksN': 5,
        'Prev_Entry': 7,
        'Prev_Entry-2': 31,
        'Mov_Avg_Nig': 19,
        'Vert_Avg_Nig': 6.5
    },
    {
        'Row Number': 1412,
        'Morning': 15,
        'Prev_Week': 27,
        '2WeeksM': 3,
        'Prev_Entry': 28,
        'Prev_Entry-2': 7,
        'Mov_Avg_Mor': 17.5,
        'Vert_Avg_Mor': 15,
        'Afternoon': 5,
        'Prev_Week': 22,
        '2WeeksA': 10,
        'Prev_Entry': 15,
        'Prev_Entry-2': 28,
        'Mov_Avg_Aft': 21.5,
        'Vert_Avg_Aft': 16,
        'Evening': 2,
        'Prev_Week': 32,
        '2WeeksE': 4,
        'Prev_Entry': 5,
        'Prev_Entry-2': 15,
        'Mov_Avg_Eve': 10,
        'Vert_Avg_Eve': 18,
        'Night': 2,
        'Prev_Week': 30,
        '2WeeksN': 6,
        'Prev_Entry': 2,
        'Prev_Entry-2': 5,
        'Mov_Avg_Nig': 3.5,
        'Vert_Avg_Nig': 18
    },
    {
        'Row Number': 1413,
        'Morning': 13,
        'Prev_Week': 20,
        '2WeeksM': 11,
        'Prev_Entry': 2,
        'Prev_Entry-2': 2,
        'Mov_Avg_Mor': 2,
        'Vert_Avg_Mor': 15.5,
        'Row Number': 1413,
        'Afternoon': 28,
        'Prev_Week': 29,
        '2WeeksA': 19,
        'Prev_Entry': 13,
        'Prev_Entry-2': 2,
        'Mov_Avg_Aft': 7.5,
        'Vert_Avg_Aft': 24,
        'Evening': 22,
        'Prev_Week': 23,
        '2WeeksE': 29,
        'Prev_Entry': 28,
        'Prev_Entry-2': 13,
        'Mov_Avg_Eve': 20.5,
        'Vert_Avg_Eve': 26,
        'Night': 12,
        'Prev_Week': 2,
        '2WeeksN': 7,
        'Prev_Entry': 22,
        'Prev_Entry-2': 28,
        'Mov_Avg_Nig': 25,
        'Vert_Avg_Nig': 4.5
    },
    {
        'Row Number': 1414,
        'Morning': 12,
        'Prev_Week': 29,
        '2WeeksM': 14,
        'Prev_Entry': 12,
        'Prev_Entry-2': 22,
        'Mov_Avg_Mor': 17,
        'Vert_Avg_Mor': 21.5,
        'Row Number': 1414,
        'Afternoon': 35,
        'Prev_Week': 7,
        '2WeeksA': 31,
        'Prev_Entry': 12,
        'Prev_Entry-2': 12,
        'Mov_Avg_Aft': 12,
        'Vert_Avg_Aft': 19,
        'Evening': 31,
        'Prev_Week': 5,
        '2WeeksE': 32,
        'Prev_Entry': 35,
        'Prev_Entry-2': 12,
        'Mov_Avg_Eve': 23.5,
        'Vert_Avg_Eve': 18.5,
        'Night': 11,
        'Prev_Week': 3,
        '2WeeksN': 18,
        'Prev_Entry': 31,
        'Prev_Entry-2': 35,
        'Mov_Avg_Nig': 33,
        'Vert_Avg_Nig': 10.5
    },
    {
        'Row Number': 1415,
        'Morning': 14,
        'Prev_Week': 25,
        '2WeeksM': 5,
        'Prev_Entry': 11,
        'Prev_Entry-2': 31,
        'Mov_Avg_Mor': 21,
        'Vert_Avg_Mor': 15,
        'Row Number': 1415,
        'Afternoon': 2,
        'Prev_Week': 9,
        '2WeeksA': 14,
        'Prev_Entry': 14,
        'Prev_Entry-2': 11,
        'Mov_Avg_Aft': 12.5,
        'Vert_Avg_Aft': 11.5,
        'Evening': 23,
        'Prev_Week': 7,
        '2WeeksE': 30,
        'Prev_Entry': 2,
        'Prev_Entry-2': 14,
        'Mov_Avg_Eve': 8,
        'Vert_Avg_Eve': 18.5,
        'Row Number': 1415,
        'Night': 25,
        'Prev_Week': 5,
        '2WeeksN': 22,
        'Prev_Entry': 23,
        'Prev_Entry-2': 2,
        'Mov_Avg_Nig': 12.5,
        'Vert_Avg_Nig': 13.5
    }
]

# Iterate through the provided data to update the corresponding columns in the DataFrame
for data in provided_data:
    row_number = data['Row Number']
    for column in data.keys():
        if column != 'Row Number':
            unseen_data.loc[unseen_data['Row Number'] == row_number, column] = data[column]

# Display the dataset after NaN handling
print("First few rows of unseen data after NaN handling:")
print(unseen_data.head())

# Check for NaN values in the entire dataset
nan_counts = unseen_data.isnull().sum()
print("Count of NaN values in unseen data:")
print(nan_counts)

# Save the updated unseen data
updated_unseen_path = os.path.join(drive_dataset_directory, '16_preprocessed_unseen_data.csv')
unseen_data.to_csv(updated_unseen_path, index=False)
logger.info("Updated unseen data saved successfully.")


First few rows of unseen data after NaN handling:
   Row Number Data_Type  Morning  Prev_Week  Rep_Prev_Week  Prev_Entry  \
0        1410    Unseen     13.0         26           26.0          26   
1        1411    Unseen     21.0          8           22.5           7   
2        1412    Unseen     15.0         30           15.0           2   
3        1413    Unseen     13.0          2           15.5          22   
4        1414    Unseen     12.0          3           21.5          31   

   2WeeksM  Mov_Avg_Mor  Afternoon  Prev_Week.1  ...  Day  Prediction1  \
0     25.0          6.0         20            7  ...    1          NaN   
1     12.0         22.0         31           18  ...    2          NaN   
2      3.0         17.5          5           22  ...    3          NaN   
3     11.0          2.0         28           29  ...    4          NaN   
4     14.0         17.0         35            7  ...    5          NaN   

   Prev_Morning  Prev_Afternoon  Prev_Evening  Vert_Avg_Mor 